In [ ]:
from google.colab import drive
import os, requests, time
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#dataset
dataset_raw_url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
base_url = "https://drive.google.com/drive/folders/1gNZcvtlRTG70LpEOZn-JShjE_3m1uaD8"
base_path="./drive/Shareddrives/AIDE/Business/Progetto"
dataset_path = os.path.join(base_path, "dataset")
dataset_path_human = os.path.join(dataset_path, "human")
dataset_path_bot = os.path.join(dataset_path, "bot")

os.listdir(dataset_path)

['human', 'bot']

# GPT INTERFACE
To send API request, it's sufficient to pick a generator name and make the proper header (as instructed in the official documentation).

In [ ]:
#GPT3.5 TURBO
SECRET_KEY_CHATGPT = "***************************************"
URL_API_CHATGPT = "https://api.openai.com/v1/chat/completions"
MODEL_CHATGPT_GENERATOR = "gpt-3.5-turbo"
REQUEST_HEADER_CHATGPT = {
      "Content-Type": "application/json",
      "Authorization": "Bearer "+SECRET_KEY_CHATGPT
  }

In [ ]:
# WARNING gpt4o has higher billing prices
#GPT4o
SECRET_KEY_CHATGPT = "***************************************"
URL_API_CHATGPT = "https://api.openai.com/v1/chat/completions"
#MODEL_CHATGPT_GENERATOR = "gpt-4o"
REQUEST_HEADER_CHATGPT = {
      "Content-Type": "application/json",
      "Authorization": "Bearer "+SECRET_KEY_CHATGPT
  }

# AUTOMATION

the generator is going to save the generated reviews directly into the drive folder of the database

In [ ]:
# List all files in the folder
files = os.listdir(dataset_path_bot)

# Count the number of files
num_files = len(files)

print("Number of files in the folder:", num_files)

Number of files in the folder: 7811


# REQUIREMENTS

To avoid discarding any reviews (since we paid for them), they are going to be requested by explicitly stating the boundaries we have decided to consider acceptable for the dataset.

This is different from the human review folder, since some of them are too long or too short, and some of them are ignored by our classificator.

In [ ]:
MIN_SIZE_WORD = 100
MIN_SIZE_CHAR = 500
MAX_SIZE_CHAR = 1800

# name template to save files
NAME_TAG = "review_26_05_GPT_3.5"

PROMPT_TEMPLATE = "Generate for me a review about a fictional movie. The review should contain a number of characters between "+str(MIN_SIZE_CHAR)+" and "+str(MAX_SIZE_CHAR)+", but you can make it as long as you want in this range. The output should be ASCII text only, with no special characters, no carriage return, no text formatting nor blank lines."
PROMPT_VARIABLE = "The title of the film is this: "
PROMPT_SENTIMENT = "Pretend the writer was "

SENTIMENT_VARIABLE = ["eager to see the sequel", "annoyed", "happy", "bored",
                      "a young boy watching the film with his family", "an expert", "a teenager"]

#SENTIMENT_VARIABLE = ["satisfied of the film", "angry because he didn't like the film", "enthusiastic to watch the film again", "bored by the film", "a young boy", "a journalist expert in cinema", "an old woman watching the film with her grand-children"]

# MAKING THE PROMPT

For each request, a different prompt is submitted. In particular, it is applied the variable part and packed as url request.

The answer is extracted as a string and saved into the database.

In [ ]:

# take a request, send to chatgpt, and return the full response
def send_prompt(t):
  # ATTENZIONE:  SI PAGA A OGNI PROMPT RICHIESTO
  data = {
      "model": MODEL_CHATGPT_GENERATOR,
      "messages": [
          {"role": "user", "content": t}
      ]
  }
  response = requests.post(URL_API_CHATGPT, headers=REQUEST_HEADER_CHATGPT, json=data)
  response_data = response.json()
  completions = response_data["choices"][0]["message"]["content"]
  return completions

# generate a fake review
def make_review(prompt):
  #prompt = produce_prompt(key, sentiment)
  response = send_prompt(prompt)

  return response

# produce a prompt with a small variance from the template
def produce_prompt(key, sentiment):
  return PROMPT_TEMPLATE+PROMPT_VARIABLE+key+". "+PROMPT_SENTIMENT+sentiment

In [ ]:
def save_review(review_text,review_name):
  file_path = dataset_path_bot+"/"+review_name+".txt"
  with open(file_path, "w") as file:
        file.write(review_text)

# FILM TITLES
Each prompt is generated from a title. In order to split the execution time in several sessions, we manually changed the title list by adding new ones between each execution.

In [ ]:
#WordList TO-DO
word_list =  [
        "Stellar Voyage",
    "Interstellar Odyssey",
    "The Dream Within",
    "Quantum Leap",
    "Infinite Journey",
    "Parallel Worlds",
    "Secret Galaxy",
    "Time Paradox",
    "Mystic Falls",
    "Lost Kingdom",
    "Legend Reborn",
    "Eternal Quest",
    "Hidden Fortress",
    "Forbidden Empire",
    "Mystery Island",
    "Enchanted Forest",
    "Forgotten Realms",
    "Shadowland",
    "Cursed Treasure",
    "Arcane Secrets",
    "Phantom Ship",
    "Silent Echoes",
    "Fallen Stars",
    "Haunted Manor",
    "Twilight Saga",
    "Fabled Legends",
    "Wild West",
    "Ocean's Mystery",
    "Frozen Journey",
    "Golden Compass",
    "Shattered Dreams",
    "Beyond the Horizon",
    "Dragon's Lair",
    "Celestial Light",
    "Savage Lands",
    "Darkness Falls",
    "Rebel Heart",
    "Final Stand",
    "Galactic Heroes",
    "Twisted Fate",
    "Sacred Ground",
    "Whispering Winds",
    "Forgotten Heroes",
    "Rising Phoenix",
    "Celestial Knights",
    "Fading Echoes",
    "Serpent's Eye",
    "Hidden Shadows",
    "Crimson Sky",
    "Desert Mirage",
    "Frozen Flames",
    "Emerald Dawn",
    "Whispering Shadows",
    "Phantom Realm",
    "Burning Skies",
    "Dragon's Fury",
    "Lost Horizon",
    "Midnight Shadows",
    "Sacred Stones",
    "Echoes of Silence",
    "Frostbite",
    "Savage Shadows",
    "Rising Sun",
    "Mystic Quest",
    "Hidden Path",
    "Final Hour",
    "Ghostly Echoes",
    "Twilight Hunt",
    "Dragon's Curse",
    "Celestial Path",
    "Forbidden Kingdom",
    "Phantom Voyage",
    "Silent Night",
    "Lost Valley",
    "Twisted Shadows",
    "Burning Horizon",
    "Crimson Dawn",
    "Shattered Light",
    "Haunted Woods",
    "Phantom Night",
    "Sacred Quest",
    "Mystic Horizon",
    "Wild Heart",
    "Rebel Shadows",
    "Fading Stars",
    "Echoing Silence",
    "Savage Quest",
    "Whispering Forest",
    "Crimson Nights",
    "Sacred Flame",
    "Rising Echoes",
    "Mystic Forest",
    "Dragon's Heart",
    "Final Horizon",
    "Lost Flames",
    "Twilight Quest",
    "Celestial Journey",
    "Fabled Echoes",
    "Midnight Journey",
    "Twisted Night",
    "Dragon's Night",
    "Golden Dawn",
    "Rebel Quest",
    "Shattered Sky",
    "Frozen Dawn",
    "Savage Dawn",
    "Fading Journey",
    "Twilight Realm",
    "Celestial Flames",
    "Dragon's Echoes",
    "Burning Night",
    "Whispering Quest",
    "Sacred Shadows",
    "Lost Night",
    "Eternal Journey",
    "Fabled Kingdom",
    "Rebel Night",
    "Wild Quest",
    "Twilight Dawn",
    "Mystic Night",
    "Savage Night",
    "Golden Night",
    "Final Night",
    "Lost Shadows",
    "Twisted Dawn",
    "Phantom Heart",
    "Shattered Night",
    "Echoing Dawn",
    "Wild Journey",
    "Savage Heart",
    "Celestial Night",
    "Burning Echoes",
    "Mystic Dawn",
    "Golden Shadows",
    "Rising Journey",
    "Sacred Journey",
    "Shattered Echoes",
    "Haunted Quest",
    "Eternal Night",
    "Fabled Journey",
    "Dragon's Journey",
    "Crimson Echoes",
    "Whispering Journey",
    "Mystic Shadows",
    "Wild Flames",
    "Twilight Heart",
    "Celestial Heart",
    "Final Quest",
    "Rising Shadows",
    "Shattered Journey",
    "Burning Journey",
    "Golden Flames",
    "Frozen Shadows",
    "Echoing Journey",
    "Phantom Journey",
    "Faded Echoes",
    "Sacred Heart",
    "Lost Journey",
    "Twisted Heart",
    "Haunted Journey",
    "Eternal Flames",
    "Mystic Heart",
    "Dragon's Shadows",
    "Crimson Journey",
    "Whispering Heart",
    "Twilight Shadows",
    "Final Flames",
    "Rising Flames",
    "Shattered Heart",
    "Burning Heart",
    "Golden Journey",
    "Frozen Heart",
    "Echoing Heart",
    "Phantom Flames",
    "Faded Heart",
    "Sacred Flames",
    "Twisted Flames",
    "Haunted Heart",
    "Eternal Shadows",
    "Mystic Flames",
    "Crimson Shadows",
    "Whispering Flames",
    "Twilight Flames",
    "Celestial Shadows",
    "Final Shadows",
    "Rising Heart",
    "Savage Flames",
    "Shattered Shadows"

]


In [ ]:
#WordList PROD
old_word_list = [
        "Interstellar",
    "Mad Max: Fury Road",
    "The Revenant",
    "The Martian",
    "Dunkirk",
    "Moonlight",
    "La La Land",
    "Birdman",
    "Spotlight",
    "The Shape of Water",
    "Get Out",
    "The Florida Project",
    "Lady Bird",
    "Manchester by the Sea",
    "Moonlight",
    "Three Billboards Outside Ebbing, Missouri",
    "Call Me by Your Name",
    "Black Panther",
    "Avengers: Infinity War",
    "Avengers: Endgame",
    "Spider-Man: Homecoming",
    "Captain America: Civil War",
    "Guardians of the Galaxy Vol. 2",
    "Thor: Ragnarok",
    "Black Panther",
    "Wonder Woman",
    "Logan",
    "Deadpool",
    "Deadpool 2",
    "Blade Runner 2049",
    "The Shape of Water",
    "The Favourite",
    "1917",
    "Once Upon a Time in Hollywood",
    "Joker",
    "Parasite",
    "Knives Out",
    "The Irishman",
    "Ford v Ferrari",
    "Jojo Rabbit",
    "Little Women",
    "1917",
    "The Lighthouse",
    "Marriage Story",
    "Uncut Gems",
    "A Quiet Place",
    "A Star is Born",
    "Crazy Rich Asians",
    "Bohemian Rhapsody",
    "The Farewell",
    "Us",
    "Joker",
    "Parasite",
    "Knives Out",
    "Spider-Man: Into the Spider-Verse",
    "Toy Story 4",
    "Frozen",
    "Frozen II",
    "Zootopia",
    "Inside Out",
    "Coco",
    "Moana",
    "Big Hero 6",
    "The Lego Movie",
    "How to Train Your Dragon",
    "Despicable Me",
    "Minions",
    "Finding Dory",
    "Cars",
    "Brave",
    "Ratatouille",
    "Up",
    "WALL-E",
    "Toy Story 3",
    "Frozen",
    "The Martian",
    "The Grand Budapest Hotel",
    "Inside Out",
    "Ex Machina",
    "Her",
    "Birdman or (The Unexpected Virtue of Ignorance)",
    "Boyhood",
    "The Theory of Everything",
    "12 Years a Slave",
    "Dallas Buyers Club",
    "Gravity",
    "The Artist",
    "The Hurt Locker",
    "Slumdog Millionaire",
    "No Country for Old Men",
    "The Departed",
    "Crash",
    "Million Dollar Baby",
    "The Lord of the Rings: The Return of the King",
    "Chicago",
    "A Beautiful Mind",
    "Gladiator",
    "American Beauty",
    "Shakespeare in Love",
    "Titanic",
    "The English Patient"
    "Galactic Odyssey: Journey Through the Cosmos",
    "Crystalline Reflections: The Aurora's Whisper",
    "Twilight Symphony: Echoes of the Eternal Night",
    "Shattered Veil: Chronicles of the Celestial Abyss",
    "Sapphire Shadows: The Enchanted Horizon",
    "Timeless Journey: Legends of the Nebula",
    "Ethereal Night: Secrets of the Phantom Realm",
    "Celestial Dreams: The Radiant Path",
    "Enchanted Abyss: Whispers of the Infinite",
    "Nebula's Call: The Mystic Adventure",
    "Luminous Echoes: The Golden Symphony",
    "Mystic Path: Beyond the Twilight",
    "Phantom Horizons: The Shimmering Light",
    "Fading Whispers: The Obsidian Saga",
    "Obsidian Dawn: The Silent Realms",
    "Silent Realms: The Glowing Galaxy",
    "Golden Galaxy: The Infinite Cosmos",
    "Infinite Cosmos: Serene Symphony",
    "Serene Symphony: The Crimson Legends",
    "Crimson Legends: Journey Through the Enchanted Skies",
    "Shattered Dreams: Pathways of the Forgotten Stars",
    "Eclipse of Shadows: Radiant Night",
    "Eternal Night: Hidden Secrets of the Crystal Dawn",
    "Forgotten Skies: The Glowing Enigma",
    "Dreams of Crystal: The Aurora's Veil",
    "Whispering Echoes: Pathways of the Enchanted Realms",
    "Silent Path: Journey Through the Mystic Light",
    "Journey Through Time: The Symphony of Shadows",
    "Symphony of Light: Radiant Adventures in the Cosmos",
    "Radiant Adventures: Reflections of the Golden Horizon",
    "Serene Reflections: Luminous Pathways Beyond",
    "Luminous Pathways: The Golden Dawn",
    "Golden Horizon: The Ethereal Journey",
    "Ethereal Journey: Infinite Reflections of the Celestial",
    "Enchanted Odyssey: The Timeless Shadows",
    "Timeless Shadows: Dawn of the Phantom Dreams",
    "Phantom Dawn: The Mystic Night",
    "Mystic Night: The Nebula's Glow",
    "Nebula's Glow: Aurora's Call Beyond",
    "Aurora's Call: The Celestial Veil",
    "Celestial Veil: The Eternal Symphony",
    "Eternal Symphony: Silent Echoes of the Abyss",
    "Silent Echoes: The Crimson Whispers",
    "Crimson Whispers: Golden Shadows of the Galaxy",
    "Golden Shadows: The Phantom Dreams",
    "Luminous Night: The Journey Through the Stars",
    "Journey Through the Stars: The Timeless Saga",
    "Timeless Saga: Legends of the Ethereal",
    "Ethereal Legends: Pathways of the Cosmic Dawn",
    "Obsidian Shadows of the Shimmering Dawn",
    "Galactic Odyssey of the Radiant Skies",
    "Crystalline Reflections of the Phantom Dreams",
    "Aurora's Whisper in the Enchanted Abyss",
    "Twilight Symphony of the Mystic Horizons",
    "The Shattered Veil of the Nebula's Call",
    "Sapphire Shadows of the Forgotten Dawn",
    "Timeless Journey Through the Serene Cosmos",
    "Whispers of the Ethereal Enigmatic Shadows",
    "Celestial Dreams of the Radiant Night",
    "Enchanted Reflections of the Golden Galaxy",
    "Nebula's Radiance in the Silent Realms",
    "Luminous Echoes of the Shimmering Veil",
    "The Mystic Path of Whispering Reflections",
    "Galactic Adventures Beyond the Crystal Abyss",
    "Fading Whispers of the Enigmatic Stars",
    "Radiant Horizons of the Obsidian Dawn",
    "Serene Symphony of the Ethereal Night",
    "Twilight Echoes of the Celestial Quest",
    "Aurora's Journey Through the Silent Abyss",
    "Shimmering Horizons of the Forgotten Veil",
    "The Enchanted Dawn of Whispering Stars",
    "Golden Reflections of the Mystic Abyss",
    "Eclipse of Shadows in the Radiant Cosmos",
    "Crimson Symphony of the Silent Dawn",
    "Luminous Horizons of the Timeless Dream",
    "Shattered Dreams of the Ethereal Path",
    "Whispers of the Nebula's Hidden Secrets",
    "Celestial Symphony of the Obsidian Night",
    "Infinite Reflections of the Aurora's Veil",
    "Phantom Adventures in the Enchanted Realms",
    "Radiant Dawn of the Whispering Echoes",
    "Mystic Twilight of the Golden Horizons",
    "Twilight Pathways of the Silent Abyss",
    "The Enigmatic Journey Through the Stars",
    "Luminous Shadows of the Crimson Dawn",
    "Whispers of the Radiant Eternal Night",
    "Celestial Dreams of the Obsidian Veil",
    "Shimmering Reflections of the Hidden Cosmos",
    "The Ethereal Symphony of Whispering Echoes",
    "Aurora's Call in the Forgotten Skies",
"Twilight Dreams of the Enchanted Path",
    "Serene Horizons of the Celestial Dawn",
    "Nebula's Symphony of the Infinite Abyss",
    "Radiant Reflections of the Whispering Stars",
    "Mystic Adventures of the Shattered Night",
    "The Enchanted Path of the Luminous Veil",
    "Timeless Shadows in the Aurora's Glow",
    "Golden Dreams of the Ethereal Journey",
    "Obsidian Echoes of the Radiant Cosmos",
    "Whispers from the Forgotten Realms",
    "Celestial Pathways of the Mystic Veil",
    "Luminous Symphony of the Shimmering Stars",
    "Crimson Adventures Beyond the Silent Dawn",
    "Radiant Skies of the Enigmatic Abyss",
    "Twilight Echoes of the Celestial Veil",
    "The Ethereal Whispers of the Aurora's Night",
    "Shimmering Dreams of the Infinite Cosmos",
    "Galactic Symphony of the Radiant Path",
    "Serene Adventures in the Enchanted Skies",
    "Mystic Shadows of the Golden Night",
    "Celestial Reflections of the Whispering Abyss",
    "Nebula's Path Through the Silent Stars",
    "Crystalline Echoes of the Timeless Dawn",
    "Radiant Twilight of the Obsidian Path",
    "Whispers of the Enchanted Symphony",
    "Luminous Horizons of the Golden Night",
    "The Ethereal Call of the Shattered Dreams",
    "Shimmering Reflections of the Celestial Veil",
    "Aurora's Echoes in the Mystic Abyss",
    "Infinite Pathways Through the Radiant Night",
    "Phantom Symphony of the Silent Dawn",
    "Nebula's Echoes of the Shimmering Cosmos",
    "Whispers of the Enchanted Night",
    "Crimson Horizons of the Timeless Abyss",
    "The Celestial Veil of Radiant Shadows",
    "Aurora's Path Through the Shimmering Skies",
    "Golden Symphony of the Infinite Echoes",
    "Radiant Reflections of the Phantom Stars",
    "Twilight Symphony of the Serene Night",
    "The Ethereal Journey of Mystic Whispers",
    "Luminous Echoes of the Golden Dawn",
    "Shimmering Shadows of the Celestial Abyss",
    "Timeless Reflections of the Radiant Cosmos",
    "Obsidian Pathways Through the Enchanted Veil",
    "Crimson Dreams of the Shattered Night",
    "Whispers of the Infinite Horizons",
    "Aurora's Symphony of the Radiant Path",
    "Ethereal Reflections of the Silent Veil",
    "Phantom Dawn of the Mystic Echoes",
    "Luminous Pathways Through the Enchanted Night",
    "Golden Echoes of the Timeless Symphony",
    "Nebula's Radiance in the Whispering Stars",
    "Celestial Symphony of the Radiant Dawn",
    "Infinite Echoes of the Ethereal Journey",
    "The Enigmatic Reflections of the Silent Abyss",
    "Shimmering Symphony of the Crimson Veil",
    "Whispers of the Aurora's Radiant Night",
    "Radiant Horizons of the Serene Echoes",
    "Mystic Pathways of the Golden Abyss",
    "Twilight Adventures in the Shimmering Skies",
    "Timeless Symphony of the Obsidian Dawn",
    "Luminous Dreams of the Celestial Path",
    "Aurora's Echoes in the Ethereal Abyss",
    "Radiant Pathways Through the Whispering Cosmos",
    "Nebula's Symphony of the Golden Night",
    "Shimmering Echoes of the Silent Stars",
    "Phantom Reflections of the Enchanted Journey",
    "Mystic Horizons of the Radiant Dawn",
    "Infinite Symphony of the Crimson Echoes",
    "Aurora's Journey to the Timeless Veil",
    "Celestial Adventures of the Whispering Skies",
    "Radiant Reflections of the Enigmatic Night",
    "Ethereal Symphony of the Luminous Abyss",
    "Shimmering Dreams of the Silent Stars",
    "Mystic Pathways Through the Aurora's Veil",
    "Golden Echoes of the Timeless Cosmos",
    "Whispers of the Radiant Symphony",
    "Luminous Horizons of the Infinite Night",
    "Twilight Symphony of the Celestial Abyss",
    "Crimson Reflections of the Enchanted Skies",
    "Aurora's Symphony of the Shimmering Veil",
    "The Silent Journey Through Radiant Stars",
    "Phantom Echoes of the Timeless Night",
    "Nebula's Reflections of the Whispering Abyss",
    "Radiant Path",
    "The Enchanted Twilight of Forgotten Legends",
    "Echoes from the Depths of Mystic Dreams",
    "Whispers of the Phantom's Eternal Call",
    "Journey Through the Celestial Shadows",
    "Radiant Horizons Beyond the Shattered Abyss",
    "Mystic Voyage into the Hidden Cosmos",
    "Silent Reflections of the Nebula's End",
    "Shimmering Dawn of the Ethereal Skies",
    "Crimson Shadows in the Aurora's Embrace",
    "Infinite Echoes of the Crystal Symphony",
    "Phantom Horizons of the Enigmatic Quest",
    "Fading Legends of the Obsidian Realms",
    "Celestial Visions of the Timeless Voyage",
    "Glimmering Whispers of the Lunar Path",
    "Mystical Waves of the Shattered Veil",
    "Eternal Radiance of the Serene Odyssey",
    "Galactic Dreams of the Hidden Spectrum",
    "Twilight Journey of the Whispering Stars",
    "Radiant Reflections of the Celestial Dawn",
    "The Enigmatic Echoes of the Phantom's Call",
    "Sapphire Dreams of the Infinite Horizon",
    "Nebula's Journey to the Shimmering Stars",
    "Whispering Paths of the Obsidian Abyss",
    "Ethereal Twilight of the Forgotten Realms",
    "Mystic Reflections of the Celestial Symphony",
    "Galactic Quest Beyond the Radiant Dawn",
    "Whispers from the Depths of Time",
    "The Hidden Symphony of Enchanted Echoes",
    "Aurora's Radiance in the Veiled Shadows",
    "Timeless Adventures of the Crystal Skies",
    "Enchanted Dawn of the Shattered Whispers",
    "The Ethereal Path of Infinite Reflections",
    "Golden Secrets of the Celestial Abyss",
    "Moonlit Quest in the Whispering Realms",
    "Serene Reflections of the Shimmering Night",
    "The Phantom's Journey into the Unknown",
    "Radiant Skies Beyond the Silent Horizon",
    "Eclipse of Dreams in the Enchanted Night",
    "Crimson Twilight of the Serene Galaxy",
    "Luminous Shadows of the Mystic Paths",
    "Shattered Echoes of the Forgotten Skies",
    "Golden Horizons of the Nebula's Dream",
    "The Silent Echoes of the Celestial Veil",
    "Mystic Symphony of the Whispering Stars",
    "Infinite Pathways Through the Ethereal Night",
    "Beyond",
    "Shattered",
    "Celestial",
    "Midnight",
    "Radiant",
    "Crystal",
    "Galactic",
    "Silent",
    "Fading",
    "Mystic",
    "Forgotten",
    "Enigmatic",
    "Stellar",
    "Glimmer",
    "Nebula",
    "Shattered Reflections",
    "Fading Shadows",
    "Mystic Tales",
    "Forgotten Echoes",
    "Enigmatic Skies",
    "Stellar Dreams",
    "Mystic Odyssey",
    "Enigmatic Reflections",
    "Mystic Adventures",
    "Silent Echoes",
    "Shattered Horizons",
    "Mystic Reflections",
    "Silent Adventures",
    "Radiant Odyssey",
    "Forgotten Horizons",
    "Silent Horizons",
    "Ethereal Odyssey",
    "Stellar Adventures",
    "Shattered Odyssey",
    "Cosmic Echoes",
    "Silent Whispers",
    "Radiant Whispers",
    "Mystic Echoes",
    "Forgotten Dreams",
    "Cosmic Whispers",
    "Galactic Echoes",
    "Enigmatic Dreams",
    "Silent Dreams",
    "Radiant Echoes",
    "Echoes of Time",
    "Radiant Skies",
    "Celestial Reflections",
    "Enchanted Voyage",
    "Mystic Whispers",
    "Enchanted Echoes",
    "Ethereal Reflections",
    "Ethereal Adventures",
    "Enchanted Reflections",
    "Radiant Reflections",
    "Stellar Reflections",
    "Cosmic Reflections",
    "Radiant Horizons",
    "Forgotten Reflections",
    "Celestial Odyssey",
    "Ethereal Dreams",
    "Journey Beyond",
    "Shadows in Time",
    "Phantom Voyage",
    "Shattered Spectrum",
    "Mystic Legends",
    "Galactic Spectrum",
    "Celestial Journey",
    "Echoes of the Cosmos",
    "Glimmering Horizons",
    "Midnight Whispers",
    "Crimson Echo",
    "Whispers of Infinity",
    "Galactic Wonders",
    "Celestial Embrace",
    "Mystic Mirage",
    "Shattered Infinity",
    "Eternal Voyage",
    "Radiant Legends",
    "Whispers of Dawn",
    "Twilight Journey",
    "Nebula's Edge",
    "Crimson Dreams",
    "Mystic Legends",
    "Cosmic Harmony",
    "Nebula's Call",
    "Enigmatic Symphony",
    "Galactic Symphony",
    "Midnight Quest",
    "The Sum of All Fears",
    "The Tale of Despereaux",
    "The Terminal",
    "The Thin Red Line",
    "The Time Traveler's Wife",
    "The Truman Show",
    "The Untouchables",
    "The Way Back",
    "The World's End",
        "The Life Aquatic with Steve Zissou",
    "The Man from Earth",
    "The Man from U.N.C.L.E.",
    "The Man Who Fell to Earth",
    "The Man Who Wasn't There",
    "The Old Man and the Sea",
    "The Old Guard",
    "The One I Love",
    "The Other Woman",
    "The Outlaw Josey Wales",
    "The Place Beyond the Pines",
    "The Princess and the Frog",
    "The Road to El Dorado",
    "The Secret Garden",
    "The Secret in Their Eyes",
    "The Secret of Kells",
    "The Secret of Moonacre",
    "The Shape of Water",
    "The Simpsons Movie",
    "The Sixth Sense",
    "The Smurfs",
    "The Sorcerer's Apprentice",
    "The Sound of Music",
    "The Spiderwick Chronicles",
    "The SpongeBob Movie: Sponge Out of Water",
    "The Spy Next Door",
    "The Spy Who Dumped Me",
    "The Squid and the Whale",
         "The Hitchhiker's Guide to the Galaxy",
    "The Man Who Knew Too Much",
    "The Theory of Everything",
    "The Imaginarium of Doctor Parnassus",
    "The Perks of Being a Wallflower",
    "The Secret Life of Walter Mitty",
    "The United States vs. Billie Holiday",
    "The Boy in the Striped Pyjamas",
    "The Diving Bell and the Butterfly",
    "The Adventures of Priscilla, Queen of the Desert",
    "The Hundred-Year-Old Man Who Climbed Out of the Window and Disappeared",
    "The Sisterhood of the Traveling Pants",
    "The Guernsey Literary and Potato Peel Pie Society",
    "The Assassination of Richard Nixon",
    "The Disappearance of Eleanor Rigby",
    "The Englishman Who Went Up a Hill But Came Down a Mountain",
    "The Men Who Stare at Goats",
    "The Last King of Scotland",
         "The Assassination of Jesse James",
    "Star Wars: The Force Awakens",
    "Indiana Jones and the Last Crusade",
    "Pirates of the Caribbean: The Curse of the Black Pearl",
    "The Hobbit: An Unexpected Journey",
    "The Day the Earth Stood Still",
    "The Fast and the Furious: Tokyo Drift",
    "Avatar",
    "Inception",
    "Titanic",
    "Gladiator",
    "Memento",
    "Amelie",
    "Joker",
    "Braveheart",
    "Parasite",
    "Whiplash",
    "La La Land",
    "Blade Runner",
    "Fight Club",
    "The Godfather",
    "Jurassic Park",
    "The Matrix",
    "Saving Private Ryan",
    "Forrest Gump",
    "Good Will Hunting",
    "The Dark Knight",
    "A Beautiful Mind",
    "Schindler's List",
    "The Social Network",
    "Back to the Future",
    "The Silence of the Lambs",
    "The Pursuit of Happyness",
    "The Wolf of Wall Street",
    "Guardians of the Galaxy",
    "The Grand Budapest Hotel",
    "The Shawshank Redemption",
    "Pulp Fiction",
    "Eternal Sunshine of the Spotless Mind",
    "The Lord of the Rings",
    "The King's Speech",
    "The Great Gatsby",
    "No Country for Old Men",
    "The Curious Case of Benjamin Button",
    "The Girl with the Dragon Tattoo",
    "Harry Potter and the Philosopher's Stone",
    "The Chronicles of Narnia",
    "The Adventures of Tintin",
    "The Eternal Knight",
"The Hidden Cavern",
"The Dream Weaver",
"The Ghostly Symphony",
"Twilight's Edge",
"The Timekeeper's Secret",
"The Mystic River",
"The Forbidden Island",
"Echoes of Atlantis",
"The Cursed Amulet",
"The Phoenix Feather",
"The Shadow Realm",
"The Hidden Temple",
"The Last Sanctuary",
"Moonshadow's Kiss",
"The Secret Passage",
"The Lost Valley",
"The Golden Empire",
"The Enchanted Rose",
"The Silent Storm",
"The Phantom's Curse",
"The Hidden Fortress",
"The Enchanted Mirror",
"The Eternal Star",
"The Serpent's Lair",
"The Haunted Crypt",
"The Last Enigma",
"The Shadow of the Moon",
"The Dreamcatcher",
"The Ghost Ship",
"The Lost Heir",
"The Mystic Cavern",
"The Celestial Sphere",
"The Phantom Dancer",
"The Timeless Voyage",
"The Hidden Gem",
"The Phantom's Lair",
"The Mystic Portal",
"The Enchanted Castle",
"The Crimson Shadow",
"The Celestial Warrior",
"The Dreamer's Quest",
"The Phantom's Whisper",
"The Forbidden Journey",
"The Silent Echo",
"The Phantom's Call",
"The Crimson Tide",
"The Phantom's Secret",
"The Eternal Night",
"The Shadow's Edge",
"The Phantom's Labyrinth",
"The Hidden Path",
"The Mystic's Call",
"The Crimson Enigma",
"The Eternal Promise",
"The Phantom's Legacy",
"The Shadow's Curse",
"The Mystic's Secret",
"The Crimson Oath",
"The Eternal Echo",
"The Phantom's Promise",
"The Shadow's Legacy",
"The Phantom Ship",
"The Eternal Quest",
"The Sapphire Sword",
"The Darkened Path",
"The Celestial Compass",
"The Haunted Lighthouse",
"The Lost Horizon",
"The Crystal Kingdom",
"The Land of the Lost",
"Song of the Sea",
"Shadows in the Mist",
"Garden of Lost Dreams",
"Journey to the Moon",
"The Last Warrior",
"The Labyrinth of Time",
"The Enchanted Forest",
"Echoes of the Past",
"Silent Whispers",
"The Forgotten Kingdom",
"A Glimpse of Eternity",
"The Hidden Oasis",
"Midnight in Paris",
"The Lighthouse Keeper",
"Ghosts of Yesterday",
"The Phoenix's Call",
"Desert Mirage",
"A Pirate's Life",
"The Crystal Cave",
"Whispering Pines",
"The Haunted Manor",
"Dance of the Fireflies",
"Wings of Freedom",
"Moonlit Serenade",
"The Winter's Tale",
"The Starship Chronicles",
"Legacy of the Titans",
"The Quantum Enigma",
"The Time Traveler's Diary",
"The Forbidden Forest",
"Emerald Isle",
"Secrets of the Sand",
"The Dragon's Lair",
"The Eternal Flame",
"Twilight Shadows",
"Mysteries of the Deep",
"The Lost Expedition",
"The Silver Lining",
"Autumn's Embrace",
"Celestial Harmony",
"The Nomad's Journey",
"The Ivory Tower",
"Under the Northern Lights",
"The Rebel's Code",
"Symphony of Dreams",
"The Whispering Wind",
"Kingdom of Ashes",
"The Guardian's Oath",
"Beneath the Waves",
"The Alchemist's Quest",
"Labyrinth of Shadows",
"The Golden Compass",
"The Crimson Crown",
"Chronicles of the Beyond",
"The Silent Fortress",
"The Fading Light",
"The Secret Garden",
"Echoes in the Canyon",
"The Forgotten Treasure",
"Journey to Atlantis",
                 "Beyond the Horizon",
"The Night of Falling Stars",
"The Secret of the Wind",
"Heart of the Storm",
        "Whispers of Serenity",
"Echoes of Harmony",
"Sunset Symphony",
"Enchanted Horizon",
"Cascade of Dreams",
"Twilight Whispers",
"Serenade by Moonlight",
"Ocean\'s Harmony",
"Butterfly Serenity",
"Symphony of Shadows",
"Whimsical Whispers",
"Tranquil Twilight",
"Whispering Meadows",
"Serendipity Sunrise",
"Radiant Reflections",
"Whispering Breeze",
"Enchanted Cascade",
"Harmony\'s Echo",
"Whispering Horizon",
"Tranquil Tranquility",
"Dreaming of Aurora",
"Harmony\'s Meadow",
"Rainbow Serenade",
"Serenity\'s Symphony",
"Butterfly\'s Whisper",
"Horizon\'s Harmony",
"Echoes of Tranquility",
"Whispering Galaxy",
"Moonlight Serenade",
"Oceanic Symphony",
"Cascade of Echoes",
"Whispering Rainbow",
"Symphony of Dreams",
"Whispering Forest",
"Serenade of Shadows",
"Tranquil Butterfly",
"Whispering Enchantment",
"Sunset Serenity",
"Harmony\'s Mirage",
"Meadow\'s Whisper",
"Serenade of Solitude",
"Cascade of Harmony",
"Whispering Enigma",
"Symphony of Whispers",
"Tranquil Enchantment",
"Butterfly\'s Dream",
"Serenade at Sunrise",
"Harmony\'s Radiance",
"Whispering Fireflies",
"Twilight Serenade",
"Symphony of Tranquility",
"Whispering Dandelion",
"Echoes of Sunset",
"Serenity\'s Whirlwind",
"Whispering Rainbow",
"Harmony\'s Cascade",
"Enchanted Meadow",
"Whispering Serenity",
"Serenade of Echoes",
"Tranquil Symphony",
"Butterfly\'s Horizon",
"Whispering Twilight",
"Serenity\'s Reflection",
"Symphony of Shadows",
"Whispering Ocean",
"Harmony\'s Radiance",
"Tranquil Serenade",
"Whispering Sunset",
"Serenade of Serenity",
"Cascade of Dreams",
"Whispering Galaxy",
"Harmony\'s Tranquility",
"Echoes of Aurora",
"Whispering Rainbow",
"Tranquil Symphony",
"Whispering Enchantment",
"Serenity\'s Serenade",
"Symphony of Shadows",
"Whispering Twilight",
"Harmony\'s Echo",
"Whispering Meadow",
"Serenade of Sunrise",
"Symphony of Serenity",
"Whispering Forest",
"Tranquil Harmony",
"Whispering Ocean",
"Serenity\'s Symphony",
"Whispering Breeze",
"Harmony\'s Mirage",
"Serenade of Shadows",
"Whispering Rainbow",
"Symphony of Dreams",
"Whispering Enigma",
"Harmony\'s Radiance",
"Tranquil Symphony",
"Whispering Fireflies",
"Serenity\'s Whirlwind",
"Symphony of Tranquility",
"Whispering Dandelion",
"Harmony\'s Cascade","Whispering Shadows",
    "The Silent Crusade",
    "Enchanted Echoes",
    "Celestial Voyage",
    "Mystic Horizons",
    "The Phantom's Call",
    "Eternal Skies",
    "Fading Stars",
    "Galactic Legends",
    "Crystal Memories",
    "Radiant Quest",
    "Beyond the Abyss",
    "Silent Dreams",
    "Shattered Realms",
    "Enigmatic Journey",
    "Forgotten Whispers",
    "Nebula's End",
    "Mystic Dawn",
    "The Silent Empire",
    "Ethereal Visions",
    "Galactic Shadows",
    "Mystic Reflections",
    "The Hidden Echo",
    "Eternal Whispers",
    "Celestial Dreams",
    "The Forgotten Odyssey",
    "Whispers of Eternity",
    "Shattered Skies",
    "Enigmatic Reflections",
    "The Silent Voyage",
    "Radiant Dreams",
    "Beyond the Stars",
    "Mystic Realms",
    "Galactic Whispers",
    "Fading Echoes",
    "Crystal Odyssey",
    "The Silent Adventure",
    "Enchanted Dreams",
    "Shattered Cosmos",
    "Celestial Symphony",
    "Eternal Reflections",
    "Nebula's Secret",
    "The Mystic Quest",
    "Whispers of Time",
    "Forgotten Realms",
    "Silent Reflections",
    "Enigmatic Whispers",
    "Galactic Echoes",
    "Radiant Horizons",
    "Beyond the Veil",
    "The Hidden Realm",
    "Mystic Whispers",
    "Celestial Reflections",
    "Shattered Horizons",
    "Eternal Mysteries",
    "Fading Dreams",
    "Galactic Adventures",
    "Silent Echoes",
    "The Enchanted Odyssey",
    "Forgotten Legends",
    "Nebula's Journey",
    "Radiant Reflections",
    "Ethereal Horizons",
    "The Silent Quest",
    "Mystic Horizons",
    "Shattered Echoes",
    "Galactic Reflections",
    "Whispers of Infinity",
    "Eternal Adventures",
    "Celestial Odyssey",
    "The Hidden Whispers",
    "Enigmatic Dreams",
    "Silent Mysteries",
    "Forgotten Reflections",
    "Radiant Echoes",
    "Mystic Legends",
    "Fading Reflections",
    "Galactic Mysteries",
    "The Silent Spectrum",
    "Shattered Adventures",
    "Ethereal Reflections",
    "Celestial Echoes",
    "Whispers of the Past",
    "Forgotten Echoes",
    "Radiant Adventures",
    "Nebula's Reflections",
    "Mystic Echoes",
    "Silent Legends",
    "Shattered Secrets",
    "Galactic Odyssey",
    "Eternal Whispers",
    "Enchanted Horizons",
    "The Silent Reflections",
    "Forgotten Adventures",
    "Mystic Skies",
    "Fading Horizons",
    "Shattered Legends",
    "Celestial Whispers",
    "Radiant Mysteries",
    "Galactic Adventures",
    "Ethereal Echoes",
    "Nebula's Mysteries",
    "Silent Dreams",
    "Whispers of Time",
    "Enchanted Reflections",
    "Mystic Whispers",
    "Forgotten Realms",
    "Radiant Echoes",
    "Shattered Mysteries",
    "Galactic Whispers",
    "Eternal Horizons",
    "The Silent Echoes",
    "Mystic Adventures",
    "Enigmatic Whispers",
    "Fading Skies",
    "Celestial Adventures",
    "Silent Reflections",
    "Forgotten Echoes",
    "Radiant Dreams",
    "Shattered Whispers",
    "Galactic Mysteries",
    "Ethereal Whispers",
    "Nebula's Adventures",
    "The Silent Odyssey",
    "Mystic Reflections",
    "Enigmatic Adventures",
    "Whispers of Eternity",
    "Forgotten Dreams",
    "Celestial Mysteries",
    "Radiant Whispers",
    "Shattered Echoes",
    "Galactic Horizons",
    "Eternal Dreams",
    "The Silent Whispers",
    "Mystic Echoes",
    "Enigmatic Legends",
    "Fading Echoes",
    "Celestial Legends",
    "Silent Adventures",
    "Forgotten Whispers",
    "Radiant Mysteries",
    "Shattered Horizons",
    "Galactic Dreams",
    "Ethereal Echoes",
    "Nebula's Whispers",
    "The Silent Horizons",
    "Mystic Legends",
    "Enigmatic Whispers",
    "Whispers of Time",
    "Forgotten Mysteries",
    "Celestial Reflections",
    "Radiant Adventures",
    "Shattered Mysteries",
    "Galactic Reflections",
    "Eternal Adventures",
    "The Silent Reflections",
    "Mystic Whispers",
    "Enigmatic Dreams",
    "Fading Reflections",
    "Celestial Adventures",
    "Silent Mysteries",
    "Forgotten Echoes",
    "Radiant Whispers",
    "Shattered Echoes",
    "Galactic Mysteries",
    "Ethereal Whispers",
    "Nebula's Adventures",
    "The Silent Skies",
    "Mystic Echoes",
    "Enigmatic Adventures",
    "Whispers of Eternity",
    "Forgotten Adventures",
    "Celestial Whispers",
    "Radiant Mysteries",
    "Shattered Reflections",
    "Galactic Horizons",
    "Eternal Whispers",
    "The Silent Reflections",
    "Mystic Adventures",
    "Enigmatic Whispers",
    "Fading Whispers",
    "Celestial Reflections",
    "Silent Mysteries",
    "Forgotten Echoes",
    "Radiant Adventures",
    "Shattered Mysteries",
    "Galactic Dreams",
    "Ethereal Reflections",
    "Nebula's Whispers"
            ]

# GENERATION

The generator is currently not working because the personal API Key has been removed, to avoid unrequested billings.

In [ ]:
# FUNZIONE A PAGAMENTO

# generate a variable to give reviews a unique filethe second version include a sentiment variable
file_count = 1
for _, _, files in os.walk(dataset_path_bot):
    file_count += len(files)


for t in word_list[1:]:

  for s in SENTIMENT_VARIABLE:
    # THROTTLING ISSUES: MUST SLEEP BETWEEN EACH REQUEST
    time.sleep(1) # SECONDS!!!!
    p = produce_prompt(t,s)

    r = make_review(p) # $$$ MONEY MONEY MONEY $$$$

    save_review(r,NAME_TAG+"_"+str(file_count))
    file_count+=1